In [70]:
import pandas as pd

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [164]:
#https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [102]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Getting the dataset

In [84]:
cdf = pd.read_csv('cdf.csv')

In [85]:
cdf.shape

(305304, 7)

In [86]:
cdf['PM'].value_counts()/len(cdf)

none             0.901269
Boris Johnson    0.053743
Theresa May      0.044988
Name: PM, dtype: float64

In [87]:
#reducing dataset from 300k to 50k (removing 90% of the none values)
cdf = cdf.drop(cdf[cdf['PM'] == 'none'].sample(frac=.93).index)

In [88]:
cdf

,MONTH_INT,MONTH_STR,USERNAME,TEXT,TWEET_PROCESSED,TWEET_CLEANED,PM
2,7,Jul,Timlagor,Labour has to promise #2ndRef. No one wants a ...,"['labour', 'promise', '2ndref', 'wants', 'soft...",labour promise 2ndref wants soft brexit many p...,none
5,7,Jul,Socialist_Chris,About the level of discourse I've come to expe...,"['level', 'discourse', 'come', 'expect', 'much...",level discourse come expect much fbpe crowd re...,none
7,7,Jul,Macetrain,Nicola Sturgeon tells Boris Johnson: Brexit of...,"['nicola', 'sturgeon', 'tells', 'boris', 'john...",nicola sturgeon tells boris johnson brexit off...,Boris Johnson
9,7,Jul,DerekJa09788684,If we ever get brexit.and Boris is looking mor...,"['ever', 'brexit', 'boris', 'looking', 'dodgy'...",ever brexit boris looking dodgy continental qu...,Boris Johnson
12,7,Jul,johnleremainer,Media have finally realised that Brexit will b...,"['media', 'finally', 'realised', 'brexit', 'di...",media finally realised brexit disaster johnson...,Boris Johnson
...,...,...,...,...,...,...,...
305238,2,Feb,SenexRecruit,Article on the practicalities of Trade Marks p...,"['article', 'practicalities', 'trade', 'marks'...",article practicalities trade marks post brexit...,none
305240,2,Feb,EmilyRoseMackay,Scotland really just prodding the open wound o...,"['scotland', 'really', 'prodding', 'open', 'wo...",scotland really prodding open wound brexit ind...,none
305264,2,Feb,DaveDemocracy,its possible his customers are stocking up kno...,"['possible', 'customers', 'stocking', 'knowing...",possible customers stocking knowing whats come...,none
305292,2,Feb,ArchHartley,"No Brexit factor, no Corbyn factor. As a float...","['brexit', 'factor', 'corbyn', 'factor', 'floa...",brexit factor corbyn factor floating voter lab...,none


In [107]:
cdf = cdf['TEXT'].tolist()

TypeError: list indices must be integers or slices, not str

In [108]:
cdf

['Labour has to promise #2ndRef. No one wants a soft Brexit -many of the people on either end prefer the opposite end to the middle. People WILL throw a tantrum with their vote over this. What Labour do NOT have to do is back Remain in that #2ndRef.',
 "About the level of discourse I've come to expect from much of the FBPE crowd regarding Brexit and Jeremy Corbyn ",
 "Nicola Sturgeon tells Boris Johnson: Brexit offers 'no democracy' for Scotland ",
 'If we ever get brexit.and Boris is looking more dodgy by the day. Will my continental quilt still work ?????',
 "Media have finally realised that Brexit will be a disaster and that Johnson is a con-artist. It's all over.",
 'Remain or Leave but*, with 2nd question: which Leave do you want: No Deal or "May\'s" Deal? So Leave is still Leave, but both Remain and Leave have to define which is best/least worst. Then we have no wriggle room later that "it wasn\'t the Brexit we chose".',
 '@BorisJohnson this better be a joke or else you and the T

### Preparing the text

In [109]:
bigram = gensim.models.Phrases(cdf, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[cdf], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [110]:
# Remove Stop Words
data_words_nostops = remove_stopwords(cdf)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [112]:
print(data_lemmatized[:1][0][:30])

['want', 'soft', 'brexit', 'many', 'people', 'end', 'prefer', 'opposite', 'end', 'middle', 'people', 'throw', 'tantrum', 'vote', 'labour', 'back', 'remain']


In [111]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [113]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [114]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.329*"people" + 0.172*"time" + 0.166*"see" + 0.135*"let" + 0.046*"post" + '
  '0.016*"death" + 0.014*"strategy" + 0.013*"former" + 0.010*"previous" + '
  '0.010*"sick"'),
 (1,
  '0.340*"think" + 0.210*"take" + 0.206*"good" + 0.038*"term" + 0.032*"future" '
  '+ 0.023*"independent" + 0.019*"fund" + 0.009*"luck" + 0.009*"drop" + '
  '0.004*"complex"'),
 (2,
  '0.147*"deliver" + 0.130*"do" + 0.097*"keep" + 0.094*"hard" + 0.069*"much" + '
  '0.067*"wait" + 0.050*"trade" + 0.043*"issue" + 0.040*"cause" + '
  '0.039*"enough"'),
 (3,
  '0.320*"tell" + 0.197*"democracy" + 0.167*"first" + 0.053*"economic" + '
  '0.000*"parliament" + 0.000*"way" + 0.000*"try" + 0.000*"win" + '
  '0.000*"theresa" + 0.000*"result"'),
 (4,
  '0.152*"ask" + 0.101*"number" + 0.101*"likely" + 0.091*"old" + '
  '0.053*"obviously" + 0.046*"send" + 0.045*"afraid" + 0.031*"pop" + '
  '0.029*"priority" + 0.020*"visit"'),
 (5,
  '0.068*"support" + 0.066*"referendum" + 0.057*"talk" + 0.056*"also" + '
  '0.055*"even"

In [137]:
lda_model.print_topics()

[(0,
  '0.329*"people" + 0.172*"time" + 0.166*"see" + 0.135*"let" + 0.046*"post" + 0.016*"death" + 0.014*"strategy" + 0.013*"former" + 0.010*"previous" + 0.010*"sick"'),
 (1,
  '0.340*"think" + 0.210*"take" + 0.206*"good" + 0.038*"term" + 0.032*"future" + 0.023*"independent" + 0.019*"fund" + 0.009*"luck" + 0.009*"drop" + 0.004*"complex"'),
 (2,
  '0.147*"deliver" + 0.130*"do" + 0.097*"keep" + 0.094*"hard" + 0.069*"much" + 0.067*"wait" + 0.050*"trade" + 0.043*"issue" + 0.040*"cause" + 0.039*"enough"'),
 (3,
  '0.320*"tell" + 0.197*"democracy" + 0.167*"first" + 0.053*"economic" + 0.000*"parliament" + 0.000*"way" + 0.000*"try" + 0.000*"win" + 0.000*"theresa" + 0.000*"result"'),
 (4,
  '0.152*"ask" + 0.101*"number" + 0.101*"likely" + 0.091*"old" + 0.053*"obviously" + 0.046*"send" + 0.045*"afraid" + 0.031*"pop" + 0.029*"priority" + 0.020*"visit"'),
 (5,
  '0.068*"support" + 0.066*"referendum" + 0.057*"talk" + 0.056*"also" + 0.055*"even" + 0.054*"plan" + 0.051*"look" + 0.051*"work" + 0.044*"

In [130]:
topic

[('brexit', 0.23570515),
 ('vote', 0.15695143),
 ('want', 0.107694246),
 ('labour', 0.060682166),
 ('make', 0.058150783),
 ('remain', 0.045672636),
 ('corbyn', 0.044508204),
 ('government', 0.036609553),
 ('many', 0.031160772),
 ('believe', 0.026234407)]

In [154]:
for idx, topic in lda_model.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, '|'.join([w[0] for w in topic])))

Topic: 19 
Words: come|expect|regard|boom|area|fake|fresh|bite|approach|though|sale|stockpile|fruit|customer|minor|veg|parliament|win|brexiteer|course|seat|way|theresa|try|farage|position|pm|change|war|candidate
Topic: 4 
Words: ask|number|likely|old|obviously|send|afraid|pop|priority|visit|arrive|way|parliament|win|change|read|try|point|seat|help|theresa|option|factor|face|libdem|result|article|run|find|push
Topic: 6 
Words: campaign|watch|sign|word|hand|alone|channel|parliament|way|try|win|painting|use|claim|farage|theresa|check|change|libdem|help|face|pm|factor|blame|stand|art|option|drug|whole|point
Topic: 3 
Words: tell|democracy|first|economic|parliament|way|try|win|theresa|result|rule|claim|pm|change|point|break|truth|face|factor|seat|option|boom|farage|maybe|great|article|use|sort|blame|help
Topic: 9 
Words: country|show|hold|money|medium|economy|waste|complete|youtube|office|build|strong|confirm|statement|undemocratic|train|german|parliament|way|try|win|change|theresa|point|fa

### Checking Coherence Score

In [106]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5834964543668295


In [159]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 5)])

pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P'])

,Topic,Word,P
0,0,people,0.328942
1,0,time,0.172384
2,0,see,0.165817
3,0,let,0.135155
4,0,post,0.045561
...,...,...,...
95,19,come,0.580221
96,19,expect,0.090442
97,19,regard,0.044762
98,19,boom,0.000013


In [163]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

/Users/ironhack/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.264945 -0.372655       1        1  15.680992
8     -0.182958  0.254008       2        1  12.069698
5     -0.143910  0.104252       3        1   8.782397
7     -0.127885  0.075870       4        1   7.820809
15    -0.103860  0.048664       5        1   6.814244
17    -0.029002  0.006810       6        1   5.277423
13    -0.060124  0.019820       7        1   5.081736
11    -0.042691  0.011486       8        1   5.056278
2     -0.020593  0.004087       9        1   4.676513
14    -0.030616  0.007161      10        1   4.588258
0      0.008059 -0.003496      11        1   4.323899
10     0.040128 -0.010358      12        1   3.589903
1      0.050791 -0.011924      13        1   3.397379
16     0.065003 -0.014157      14        1   3.138333
18     0.039112 -0.009993      15        1   3.136982
9      0.093720 -0.017759      16        1   2.409076
3      0.170929 -0.022884      17        1   1.200856
6      0.177007 -0.022983      18        1   1.095221
4      0.185050 -0.022981      19        1   1.073202
19     0.176786 -0.022969      20        1   0.786791, topic_info=     Category          Freq    Term         Total  loglift  logprob
1     Default  19590.000000  brexit  19590.000000  30.0000  30.0000
35    Default  13866.000000    deal  13866.000000  29.0000  29.0000
40    Default  12210.000000     may  12210.000000  28.0000  28.0000
13    Default  13045.000000    vote  13045.000000  27.0000  27.0000
7     Default   7539.000000  people   7539.000000  26.0000  26.0000
...       ...           ...     ...           ...      ...      ...
860   Topic20      0.054462  better      1.712122   1.3970 -11.2459
977   Topic20      0.054464     way      1.713082   1.3965 -11.2458
690   Topic20      0.054462      pm      1.712389   1.3968 -11.2459
1393  Topic20      0.054462    read      1.712235   1.3969 -11.2459
973   Topic20      0.054462  change      1.712605   1.3967 -11.2459

[884 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
172       6  0.997969    accept
496       1  0.996852   achieve
105       8  0.998284  actually
293      19  0.993185    afraid
342      12  0.999112     agree
...     ...       ...       ...
539       8  0.998560     wrong
231      15  0.999446      year
134       2  0.998157       yet
338       4  0.991748     young
64       16  0.996217   youtube

[361 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 9, 6, 8, 16, 18, 14, 12, 3, 15, 1, 11, 2, 17, 19, 10, 4, 7, 5, 20])